# Семинар 2. Элементы обработки сигналов / изображений. Связь между преобразованиями Фурье и преобразованиями Радона

In [1]:
import numpy as np
import tomopy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sem2.sem2utilities import radon2d, adjradon2d
from numpy.fft import fft, fftshift, ifft, ifftshift, fftn, ifftn
import time

### Содержание

<p><div class="lev1"><a href="#1.-Введение"><span class="toc-item-num"></span>1.Введение</a></div>
<p><div class="lev1"><a href="#2.-Элементы-теории-обработки-сигналов"><span class="toc-item-num"></span>2. Элементы теории обработки сигналов</a></div>
<p><div class="lev1"><a href="#Задание-2.-Дуальное-преобразование-Радона"><span class="toc-item-num"></span>Задание 2. Дуальное преобразование Радона</a></div>
<p><div class="lev1"><a href="#Задание-3.-Матрицы-преобразований-Радона"><span class="toc-item-num"></span>
Задание 3. Матрицы преобразований Радона
</a></div>
<p><div class="lev1"><a href="#Задание-4.-Обращение-преобразований-Радона-при-помощи-матричных-представлений.-Стабильность-восстановлений"><span class="toc-item-num"></span>
    Задание 4. Обращение преобразований Радона при помощи матричных представлений. Стабильность восстановлений
</a></div>
<p><div class="lev1"><a href="#Задание-5.-Стабилизация-при-помощи-SVD"><span class="toc-item-num"></span>
Задание 5. Стабилизация при помощи SVD</a></div>
<p><div class="lev1"><a href="#Дополнительные / Домашние задания"><span class="toc-item-num"></span>Дополнительные / Домашние задания</a></div>


**Важно:** для данного семинара вам понадобится ваш индивидуальный номер, который вам раздаст лектор в начале семинара. 

In [1]:
# set id (will be used later to download binary data)
my_id = 1

## 1. Введение

Классические преобразования Радона имеют прямую связь с преобразованием Фурье, которое очень хорошо изучено 
и является очень мощным инструментом анализа во многих областях математики. 

Вышеупомянутая связь полностью описывается проекционной теоремой из курса, которую мы снова приводим здесь ниже. 
Помимо чисто математических следствий из данной теоремы (например, инъективность $R$ для классов функций $S(R^n), \mathcal{E}'(R^n)$) есть ещё одно очень важное следствие, а именно **формула обращения для** преобразования $R$. 

В курсе мы увидим ещё не один раз, что обращение $R$ можно реализовывать самими различными способами, где 
у каждого будут свои преимущества и недостатки, которые мы будем стараться обсуждать. 
Начнем мы с самого простого способа - а именно, с обращения $R$ при помощи преобразования Фурье. 


Напомним некоторые определения и математические факты из курса.


Классическое преобразование Радона $R$ на $R^2$ определяется по следующей формуле:
\begin{equation}\label{radon-def}
    Rf(s,\theta) = \int\limits_{-\infty}^{+\infty} f(s\theta+ t\theta^\perp) \, dt, 
    \, s\in R, \, \theta\in S^1.
\end{equation}

Преобразование Фурье функций на $R^2$ и на $R\times S^1$ определяются следующим образом:

   * \begin{equation}
        \widehat{f}(\xi) := \int\limits_{R^2}f(x)e^{-2\pi ix\xi}\, dx
      \end{equation} 
    
   * \begin{equation}
       \widehat{g}(\sigma, \theta) := \int\limits_{R}g(s,\theta)\, e^{-2\pi i\sigma s}\, ds
     \end{equation}


**Проекционная теорема.** Пусть $f\in \mathcal{S}(R^2)$, тогда верна следующая формула:
\begin{equation}
    \widehat{Rf}(\sigma, \theta) = \widehat{f}(\sigma\theta), \, \sigma \in R, \, \theta\in S^1.
\end{equation}

**Формула обращения Радона на плоскости.** При помощи проекционной теоремы, можно восстановить $f$ из $Rf$ используя обратное преобразование Фурье:

\begin{equation}
    f(x) = \int\limits_{S^1}d\theta \int\limits_{-\infty}^{+\infty} \dfrac{|\rho|}{2}\widehat{Rf}(\rho, \theta)
    e^{2\pi i\rho(x,\theta)}\, d\rho, \, x\in R^2.
\end{equation}
На самом деле эта формула эквивалента известной формуле Радона:

\begin{equation}
    f(x) = \dfrac{1}{4\pi}\int\limits_{S^1}\mathcal{H}[\dfrac{d}{ds}Rf](x\theta,\theta) \, d\theta
\end{equation}
Для наших вычислений мы будем пользоваться первым вариантом, поскольку он не содержит операции 
численного дифференцирования и считается более устойчивым.


В чем выигрышь такой формулы обращения? Из предыдущего семинара мы знаем, что преобразования Радона можно обращать матричными методами, но тогда мы столкнулись с двумя проблемами: 

   1. Проблема стабильности - матрица $R$ сингулярна и её обращение численно нестабильно 
      (число обусловленности может быть равно $+\infty$ с точки зрения машинной точности). Даже в случае
      незашумленных данных (содержащих лишь ошибки вычислений), качество восстановлений было относительно низким.
      
      
   2. Размер матрицы $R$ быстро растет даже для небольшого размера изображений (размер матрицы равен $N^2\times N_\theta \times N_s$ для изображения $N\times N$ пикселей, $N_\theta$ - число проекций, $N_s$ - число линий в одной проекции). Таким образом, даже для небольших задач возникает практическая проблема работы с большой матрицей, которая потенциально не вмещается в оперативную память. (Тут можно конечно привести аргумент, что матрица $R$ является разреженной и для её хранения можно использовать структуры данных, которые занимают мало памяти. Это так, но с другой стороны использование таких структур замедляет многие операции с такой матрицей (умножения, транспонирования и т. д.). В  итоге придется использовать специальные алгоритмы умножений и других операций для разреженных матриц, что требует некоторой тонкой настройки вычислений. В реальных же приложениях матрица $R$ настолько велика, что её никогда не вычисляют полностью, но работают с ней лишь по-частям.)
   
**Итого:** Преимущество формулы обращения Радона состоит в том, что это лишь линейное преобразование для которого не нужно хранить всю матрицу $R$ - таким образом исчезает проблема нехватки памяти. Также, формула Радона очень просто записывается через преобразование Фурье - что означает, что на практике можно применять алгоритм DFT (Discrete Fourier Transform) - FFT, iFFT, которые работают очень быстро, что несомненно важно для практики.  Проблема стабильности обращения $R$ сохраняется и не зависит от метода обращения, мы покажем это в дальнейшем на лекциях.

В данном задании мы начнём немного "издалека" и постепенно перейдем к более практическим вещам:

   1. Вспомним некоторые аспекты обработки дискретных сигналов, а также использование дискретного преобразования Фурье DFT и свойств FFT,  iFFT
   
   
   2. Применим теорию из первого пункта, чтобы реализовать формулу обращения Радона и применим её на практике.
      Ожидается, что она будет работать намного быстрей чем алгоритмы из прошлого задания; также теперь не будет проблемы нехватки памяти)
      
      
   3. Если успеем, посмотрим какие вопросы возникают при рассмотрении некоторых реалистичных задач из томографии (например невозможность применять методы DFT напрямую).


## 2. Элементы теории обработки сигналов

Обыкновенно обработка сигналов начинается с того, что на компьютере уже имеется некоторое дискретное представление сигнала и наша задача состоит в том, чтобы соответствующим образом обработать этот сигнал. Например,

* применить некоторое геометрическое преобразование 
* очистить от сигнал от построннего шума
* провести спектрально-частотный анализ 
* и т. д.

В отличие от изображений, которые генерируются от начала и до конца на компьютере, сигналы обычно описывают некоторую реальную сцену или объект, имеющие континуальную структуру, а дискретный сигнал в свою очередь сгенерирован некоторой оптической системой. Таким образом возникает прямой оператор $S$ (прямое преобразование), который преобразует реальную сцену, описываемую например функциями из некоторого бесконечномерного гильбертового  пространства, в конечномерное дискретное пространство. В качестве примера последнего пространства можно  взять пространство изображений размера $N \times N$ пикселей. 

Свойства оптической системы и связь между записываемым сигналом и реальной сценой содержится в операторе $S$. Поэтому, для корректной обработки сигналов в каждой конкретной задаче необходимо определить $S$
и учесть его свойства. 

Мы начнем с основных определений и принципов для оптических систем и их сигналов. Основная же часть материала в данном задании связана с преобразованием Фурье, которое мы будем постоянно использовать в курсе.

#### Определения

Так как мы оперируем в основном изображениями, то сигналы которые мы будем рассматривать будут двухмерными или трёхмерными. Размерность сигнала можно хоть и не строго, но достаточно удобно определить, как количество независимых переменных от которых он зависит. Например, 

* черно-белое изображение $I(x,y)$ - это двумерный сигнал

* черно-белый фильм - изображения сменяющиеся во времени $I(x,y, t)$ - трехмерный сигнал

* черно-белое объемное изображение (volumetric image) $I(x,y,z)$ - трёхмерный сигнал

**Определение 2.1.** Сигналом называется скалярная функция от двух переменных $f(x,y)$, либо трёх переменных  $f(x,y,z)$. Множество $D\subset R^2$ для которого определена функция $f$ называется **областью определения**, множество принимаемых значений $f(D)$ называется **множеством значений функции $f$**.

**Определение 2.2.** Сигнал называется **непрерывным**, если он определен на всем $R^2$, либо на некотором открытом подмножестве $R^2$. Сигнал называется **дискретным**, если он определен на всех парах $(n,m)\in \mathbb{Z}^2$, либо на некотором подмножестве $\mathbb{Z}^2$. В последнем случае, как правило сигнал $g[n,m]$ соответствует значению сигнала в некоторой точке $(x_n, x_m)$, которая называется **пикселем**. Довольно часто $x_n = \Delta_X n, \, x_m = \Delta_Y m$. (*Важное замечание о дискретном сигнале: значение в пикселях это не то же самое, что сигнал равен 0 вне целых координат $(n,m)$!*)

#### Понятие оптической системы

<img src="../seminar-materials/seminar-2/imaging_system.png" width="936" heigth="250" align="center">

**Определение 2.2.** Оптическая система - это система, которая принимает на вход 2D-сигнал и на выходе выдает 2D-сигнал. (нас интересуют системы 2D-2D, хотя вполне могут существововать оптические системы 3D-3D и 3D-2D)

Исходящий можно представить как
\begin{equation}
    g = S(f), \, S : L^2(R^2) \rightarrow L^2(R^2).
\end{equation}

Здесь мы рассматриваем лишь системы, которые оперируют сигналами с **конечной энергией** т. е. $\|f\|_{L^2} < +\infty$.

Какие системы обычно возникают на практике? Наиболее часто - это системы *размытия*, т. е. оператор $S$ является в некотором роде сглаживающим / усредняющим входящий сигнал.  


Оптические системы можно классифицировать по следующим категориям:
 
Амплитудные характеристики:
  * линейность - $S(\alpha f + \beta g) = \alpha S(f) + \beta S(g)$
  * стабильность - если $|f(x,y)| \leq M_f$, то $\exists M_g$, $|g(x,y)| \leq M_g < +\infty$
  * обратимость - существует преобразование $S^{-1}$, такое, что $S^{-1}(S(f)) = f$ для любой функции $f$.

Пространственные характеристики:
  * казуальность (casuality - cause) - если сигнал изменяется во времени, т. е. $f(x,y, t)$, то величина $g(x,y, t_0)$ определеяется 
    величинами $f(x,y,t), \, t < t_0$.
  * сепарабельность - $g(x,y) = S_x \circ S_y f$
  * память - если $g(x,y)$ зависит от значения $f$ только в точке $(x,y)$, то такая система называется 
    **без сохранения памяти** (например, $g(x,y) = f(x,y)^2$)
  * инвариантные относительно сдвигов - $g(x-x_0, y-y_0) = S(f_{(x_0, y_0)})(x,y), \, f_{(x_0, y_0)}(x,y) = f(x-x_0, y-y_0)$.
  * ротационно инвариантные - см. предыдущий пункт, относительно поворота плоскости
  
Нас конкретно интересуют системы, которые линейны и инвариантны относительно сдвигов (LSI - linear shift invariant systems). Далее для краткости мы будем их называть LSI-системами.

#### LSI - системы, функция PSF 

Используя [теорему Шварца о ядре](https://en.wikipedia.org/wiki/Schwartz_kernel_theorem) несложно показать, что система $S$, которая линейна и инвариантна относительно сдвигов действует на сигнал $f$ как конволюция с некоторым ядром $h$:

\begin{equation}
    g(x,y) = \int\limits_{R}\int\limits_{R} f(x', y')h(x-x', y-y')\,  dx' dy' = f \ast h (x,y).
\end{equation}

Получается, что $h(x,y)$ - это "ответ" который сгенерирует система $S$ на дельта-импульс Дирака $\delta(x-x_0, y-y_0)$. 

Функция $h$ называется **PSF** - point spread function и она полностью определяет поведение линейной системы $S$.
В оптических системах, которые встречаются на практике, обычно PSF не инвариантна относительно сдвигов. Тем не менее, если в интересующей нас области изображения PSF меняется слабо при сдвигах, LSI-системы являются удобным математическим приближением. 

Как мы уже говорили, PSF описывает "размытие" точечного сигнала, и характер размытия по сути определяет  разрешающую способность нашей оптической системы (минимальное расстояние между двумя источниками, когда мы ещё можем их различить на изображении). Интуитивно, если PSF убывает медленно, то мы получаем сильное размытие и слабую разрешающую способность, и наоборот, чем ближе PSF к $\delta$-функции Дирака, тем выше разрешающая способность.

На практике, имея PSF, разрешающую способность системы можно определеить по правилу **Рэлея**:
   * Два источника считаются "едва различимыми" ... 



С точки зрения преобразования Фурье, систему $S$ можно полностью описать функцией $H(u,v)$ определяемую формулой:
\begin{equation}
    H(u,v) = \int\limits_{-\infty}^{+\infty}\int\limits_{-\infty}^{+\infty}h(x,y)e^{-2\pi i (ux + vy)}\, dx\,dy,
    \, H = \mathcal{F}h, 
\end{equation}

где $\mathcal{F}$ - преобразование Фурье функций на $R^2$.

LSI-системы обладают важным свойством, относительного того, как они преобразуют плоские волны

**Свойство 1.** Комплексные плоские волны $e^{-2\pi i(ux + vy)}$, $u\in R, v\in R$, являются собственными функциями линейной инвариантной системы, 
с собственными значениями $H(u,v):$
\begin{equation}
    e^{-2\pi i(ux + vy)} \xrightarrow{S} H(u,v) e^{-2\pi i (ux + vy)}
\end{equation}

**Доказательство:** Запишем действие $S$ на комплексную волну при помощи преобразования Фурье:

\begin{align}
    S(e^{-2\pi i(ux + vy)}) &= e^{-2\pi i(ux + vy)} \ast h \\ 
    &= \int\limits_{-\infty}^{+\infty}\int\limits_{-\infty}^{+\infty}
    e^{-2\pi i(ux' + vy')} dx' dy' \int\limits_{-\infty}^{+\infty}\int\limits_{-\infty}^{+\infty}
    H(u', v')e^{2\pi i(u'(x-x') + v'(y-y')}\, du' \, dv' \\
    &= \int\limits_{-\infty}^{+\infty}\int\limits_{-\infty}^{+\infty} H(u', v') du' dv'
    e^{2\pi i(u'x + v'y)}
    \int\limits_{-\infty}^{+\infty}\int\limits_{-\infty}^{+\infty} e^{-2\pi i (x'(u-u') + y'(v-v'))}dx'\,dy'\\
    &= \int\limits_{-\infty}^{+\infty}\int\limits_{-\infty}^{+\infty} H(u', v') du' dv'
    e^{2\pi i(u'x + v'y)} \delta(u-u', v-v')\, du'\,dv' \\
    &= H(u,v)e^{-2\pi i(ux + vy)}.
\end{align}
Доказательство завершено.

У LSI-систем может быть намного больше собственных функций, например, для $S = \mathrm{Id}$ - любые функции будут собственными. Комплексных экспонент вполне достаточно, для представления любой функции из $L^2(R)$, к тому же  система $S$ через $H$ - действует "диагонально" в этом базисе. 


**Следствие 1.** Из Свойства 1 мы получаем важное практическое следствие. Пусть например для некоторой пары $(u_0, v_0)$, $H(u_0, v_0) = 0$. Тогда система $S$ преобразует плоскую волну $e^{-2\pi i(u_0x + v_0 y)}$ в 0 (отсутствие ответа), соответственно плоскую волну "мы не заметим". И наоборот, если $H(u_0, v_0)$ велико по сравнению с другими значениями из образа, то это значит, что система "усиливает" плоскую волну с данными параметрами. Если наш входной сигнал представим некоторой суперпозицией плоских волн, то в силу линейности системы некоторые части сигнала мы можем "потерять", а некоторые наоборот зарегистрировать с большей мощностью. Именно поэтому так важно исследовать спектральную функцию $H$, для каждой конкретной оптической системы.


**Задача 1. (факультативно)** Покажите, что преобразование $R$ на двумерной плоскости - это не LSI система. 
Рассмотрите действие $R$ на функциях с компактным носителем в квадрате $[-1,1]^2$, где образ определен в квадрате $[-1,1] \times [0, 2\pi]$.

**Задача 2.* (факультативно)** Докажите, что $R^*R$ - это LSI система. Найдите соотвтетствующие функции $h$, $H$.


#### Фильтры

В инженерной литературе функцию $H$ называют (спектрально-частотным) "фильтром". Приведём несколько часто-встречающихся примеров фильтров:
   1. \begin{equation}
           H(u,v) = \begin{cases}
               1, \, \sqrt{u^2 + v^2} \leq \nu_{Nyq}, \\
               0,  \text{ иначе.}
                \end{cases}
      \end{equation}
   
      Этот фильтр называется "идеальным", 
      так как он сохраняет все спектральные характеристики сигнала вплость до частоты Найквиста (частоты, которая 
      соответствует максимальному разрешению нашей оптической установки, см. также [теорема Шеннона-Найквиста-Котельникова](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem))
   
   2. \begin{equation}
           H(u,v) = \begin{cases}
                 \dfrac{1}{2}\left(1 + \cos(\pi \frac{\sqrt{u^2 + v^2}}{\nu_{Nyq}})\right),
                 \sqrt{u^2 + v^2}\leq \nu_{Nyq}, \\
                 0, \text{ иначе. }
                    \end{cases}
       \end{equation}   
       Этот фильтр называется "приподнятым косинусом" и, как видно из определения, он сохраняет низкие частоты 
       сигнала, т. к. $H(u,v) \approx 1.0, \, (u,v) \approx (0,0)$, а высокие понижает в два раза 
         $H(u,v) \approx 1/2, \, \sqrt{u^2 + v^2} \approx \nu_{Nyq}$. 
         
         
   3. \begin{equation}
          H^{-}_R(u,v) = \begin{cases}
              1, \, \sqrt{u^2 + v^2} \leq R, \\
              0, \text{ иначе. }
              \end{cases}
      \end{equation}
       
      Это низкочастотный фильтр, при его применении в сигнале сохранятся детали размерами не более 
      $\dfrac{\pi}{R}$.
   
   4. \begin{equation}
          H^{+}_R(u,v) = \begin{cases}
              1, \, \sqrt{u^2 + v^2} > R, \\
              0, \text{ иначе. }
              \end{cases}
      \end{equation}
   
      Это высокочастотный фильтр, при его применени в сигнале сохранятся лишь детали размерами 
      менее $\pi / R$. Такие фильтры например интересны для нахождения и усиления границ на изображении. 

Существует и множество других фильтров используемых не только для сглаживания. Такие мы ещё увидим позже в курсе.


Разберемся, как действуют вышеперечисленные фильтры.

**Пример.** Рассмотрим черно-белое изображение, полученное при помощи магнитно-резонансной томографии. Оно представляет собой матрицу размера $N\times N$, $N=512$. Применим к изображению два разных фильтра, высоко-частотный и низкочастотный (см. пример выше), где параметр выберем $R = \dfrac{\nu_{Nyq}}{8}$. Будем считать, что данное изображение соответствует квадрату $[-1,1]\times [-1,1]$, т.е. длина стороны квадрата равна 2.0. 

С помощью этих параметров можно определить частоту Найквиста:

\begin{equation}
    \nu_{Nyq} = \dfrac{\text{sampling rate}}{2} = \dfrac{\text{number of pixels}}{2\cdot\text{length of sampling interval}}.
\end{equation}

In [ ]:
# Code for the examle, read the code and run the cell

# 1. read image in numpy array

filename = './sem2/mri-fourier-example.jpg'
im = np.array(mpimg.imread(filename))
npixels = im.shape[0]                    # pixels per dimension (square image [-1,1] x [-1,1])
sampling_rate = npixels / 2.0
nyq = sampling_rate / 2.0                # Nyquist-frequency - half of the sampling rate
df  = sampling_rate / npixels            # frequency step

# 2. prepare low-frequency, high-frequency-filters
LF = np.zeros((npixels, npixels)) # low-frequency filter
HF = np.zeros((npixels, npixels)) # high-frequency filter

freq_array = fftshift(range(npixels)).astype(np.float)
freq_array[freq_array >= (npixels / 2)] -= npixels
freq_factor_x, freq_factor_y = np.meshgrid(freq_array, -freq_array)
freq_factor_x *= df
freq_factor_y *= df

RF = np.sqrt(freq_factor_x**2 + freq_factor_y**2)
LF[RF < (nyq/8.0)] = 1.0 # low frequency filter
HF[RF > (nyq/8.0)] = 1.0 # high frequency filter

# 3. apply filters in Fourier domain and take inverse

LF_im = np.abs(ifftn((ifftshift(LF) * fftn(im)))) # Fourier integrals via DFT (next section)
HF_im = np.abs(ifftn((ifftshift(HF) * fftn(im))))

fig, axs = plt.subplots(1, 3, figsize=(12, 5), sharey=True)
axs[0].imshow(im)
axs[0].set_title('original')
axs[1].imshow(LF_im)
axs[1].set_title('low-frequency part')
axs[2].imshow(HF_im)
axs[2].set_title('high-frequency part')
plt.show()

**Задача 2.1.** Используя код и картинку из примера выше, реализуйте идеальный фильтр и примените его. С теоретической точки зрения, какое изображение вы должны получить в итоге после применения этого фильтра? Получаете ли вы ожидаемое изображение? 

In [ ]:
# YOUR CODE HERE



**Ответы на вопросы из задачи 2.1:** ВАШ ОТВЕТ

**Задача 2.2.** Используя код из примера, реализуйте фильтр "приподнятый косинус" и проверье его на изображении из примера. С теоретической точки зрения, какое изображение вы должны получить в итоге после применения этого фильтра? Получаете ли вы данное изображение - прокомментируйте то, что вы видите? 

In [ ]:
# YOUR CODE HERE


**Ответы на вопросы из задачи 2.2:** ВАШ ОТВЕТ

**Задание 2.3.** Постройте изображение PSF для идеального фильтра и для приподнятого конуса. (Подсказка: PSF - это "отклик" на сигнал точечного источника)

In [ ]:
# YOUR CODE HERE


**Задача 2.3.* (факультативно)** Попробуйте придумать фильтр, который будет не только сохранять границы изображения, но и "усиливать" их (низкочастотную часть изображения можно "убрать" - лишь бы границы было лучше видно). Объясните ниже словами, как вы строите данный фильтр и почему вы считаете, что он будет усиливать границы. 

In [56]:
# YOUR CODE HERE


**Ответы на вопросы из задачи 2.2:** ВАШ ОТВЕТ

#### Преобразование Фурье при помощи DFT

В данной секции мы разберёмся как приближенно вычислять интеграл преобразования Фурье при помощи DFT. 
Удобство этого заключается в том, что DFT можно вычислить быстро при помощи FFT. Мы рассмотрим лишь одномерный случай, для старших размерностей вывод будет аналогичным.

Напомним, что одномерное преобразование Фурье определяется следующей формулой:

\begin{equation}
    G(u) = \mathcal{F}g(u) = \int\limits_{-\infty}^{+\infty} g(x)e^{-2\pi i u x}\, dx.
\end{equation}

Интеграл выше можно аппроксимировать конечной суммой, симметричной относительно $x = 0$:

\begin{equation}
    G(u) \approx \overline{G}(u) = \Delta_x \sum\limits_{n=-N/2}^{N/2-1} g(n\Delta_x) e^{-2\pi i un\Delta_x}
\end{equation}

Очевидно, что даже, если $g$ имеет неограниченный носитель, $N$ - выбирается конечным. В последнем случае, выбирается $x_{max}$, такое, что $g(x)\approx 0$ для $|x| > x_{max}$, тогда $N$ и $\Delta_x$ выбираются так, чтобы было выполнено:

\begin{equation}
    N\Delta_x \geq 2x_{max}.
\end{equation}

Сумму выше невозможно вычислить для каждого $u$, часто вполне достаточно это сделать на равномерной сетке с некоторым шагом $\{k\Delta_u : k = -K/2, \dots, K/2 -1\}$. В таком случае конечная сумма будет выглядеть следующим образом:

\begin{equation}
    \overline{G}(k\Delta_u) = \Delta_x \sum\limits_{n=-N/2}^{N/2-1}g(n\Delta_x)e^{-2\pi i kn\Delta_x\Delta_u}.
\end{equation}

Все вычисления можно провести асимптотически за $O(KN)$ flops, что слишком много для $K, N$ возникающих на практике. Для ускорения можно использовать алгоритм FFT, который можно использовать, если выполнено условие:

\begin{equation}
    \Delta_x\Delta_u = \dfrac{1}{L}, \text{ где } L - \text{ некоторое целое число.}
\end{equation}

В таком случае, для вычисления достаточно использовать $L$-точечный алгоритм DFT (можно использовать zero-padding (добавление нулями), если $L > N$). 

Достаточно часто (но не всегда) можно взять $L = K = N$. Тогда последняя сумма превращается в "почти полную" сумму, возникающую в DFT:

\begin{equation}
    G[k] = \Delta_x \sum\limits_{n=-N/2}^{N/2-1}g(n\Delta_x)e^{-2\pi ikn/N}, \, k = -N/2, \dots, N/2-1.
\end{equation}

Данная сумма "почти полностью" соответствует DFT, в том смысле, что здесь суммирование ведется от $-N/2$ до $N/2-1$, в то время как в DFT она берется от $0$ до $N-1$. Но в силу того, что $e^{-2\pi i k(n+N)} = e^{-2\pi i kn}$ для всех $k,n$, её можно записать через DFT. Проверка остается читателю в качестве упражнения, получается лишь следующая итоговая формула:

\begin{align}
    G[k] &= \Delta_x \sum\limits_{n=0}^{N-1} \overline{g}[n]e^{-2\pi ikn/N} = DFT(\overline{g})[k], \, 
    k = -N/2, \dots, N/2-1,\\
    \overline{g}[n] &= \begin{cases}
        g[n], n = 0, \dots, N/2-1,\\
        g[N-n], n = N/2, \dots, N-1.
    \end{cases} = \text{ifftshift}(g)[n].
\end{align}

Функции `DFT` и `ifftshift` имплементированы в NumPy и во многих других библиотеках. Последнее замечание касается того, что `DFT` вычисляет суммы для $k=0,\dots, N-1$, а нас интересуют частоты симметричные относительно $u=0$.


Используя аналогичный аргумент периодичности, получаем следующий алгоритм вычисления одномерного преобразования Фурье при помощи DFT:

    # 1D Fourier transform via FFT
    
    G[k] = delta_x * fftshift[DFT[ifftshift[g]]][k], k = -N/2, ... , N/2-1. 
    
**Замечание:** Стоит отметить, что во всех наших вычислениях мы предполагали, что $N$ - чётное. Если это не выполнено, тогда функции `fftshift, ifftshift` работают по-разному и это необходимо отдельно учитывать. 

**Задание 2.1.** Вычислите численно преобразование Фурье от функции $g(x,y) = 1 / (x^2 + y^2 + 1)^{3/2}$. Сравните результат с аналитическим результатом, посчитав его заранее. 

In [ ]:
# parameters of the domain

nx = 2**8
ny = nx
xmax = 2**3
dx = 2*xmax / nx
dy = dx

# create g(x,y)
x = np.linspace(-nx/2, nx/2-1) * dx
y = np.linspace(-ny/2, ny/2-1) * dy
xx,yy = np.meshgrid(x, y)
gxy = 1. / (xx**2 + yy**2 + 1)**(1.5)

# YOUR CODE HERE 


## 3. Обращение преобразований Радона при помощи преобразования Фурье и алгоритма обратной проекции

В данном параграфе мы займемся реализацией формулы обратной проекции (формулу обращения Радона) через обратное преобразования Фурье при помощи DFT.

Напомним формулу обращения, полученную во введении:

\begin{equation}
    f(x) = \int\limits_{S^1}d\theta \int\limits_{-\infty}^{+\infty} \dfrac{|\rho|}{2}\widehat{Rf}(\rho, \theta)
    e^{2\pi i\rho(x,\theta)}\, d\rho, \, x\in R^2.
\end{equation}

Данную формулу можно приблизить следующей конечной суммой:

\begin{equation}
    f(x) \approx \Delta_\theta \Delta_\rho\sum_{i=1}^{N_\theta} \sum_{j = 1}^{N_\rho} \dfrac{|\rho_j|}{2}
    \widehat{Rf}(\rho_j, \theta_i) e^{2\pi i (x,\theta_i)\rho_j}, 
\end{equation}

где $\widehat{Rf}(\rho_j,\theta_i)$ - одномерное преобразование Фурье от $Rf$, $\Delta_\rho, \Delta_\theta, N_\rho, N_\theta$ - параметры семплирования лучевых данных.

Таким образом псевдо-алгоритм обращения $R$ при помощи формулы обратной проекции следующий:

    # BEGIN 
        
        # initilializittion
        0. Delta_phi, Delta_shift, N_shift, N_theta
        1. initialize arrays: shift[i], theta[j], rho[j]
        2. initialize domain array : x[k] 
           values of the function  : f[k]
        3. padding_length
        
        # compute Rf^(rho_j, theta_i) via 1D Fourier transforms via DFT 
        
        for all i do : 
            Rf(s, theta_i) <- padd with zeros on both sides along s-axis
            Rf^(rho_j, theta_i) = fftshift(fft(ifftshift(Rf(* , theta_i))))
        endfor
        
        # compute values of f[k] at x[k] via approximation of FBP
        
        for x in x[k] do : 
            f[k] = Delta_rho Delta_phi * sum_i sum_j (rho_j / 2) * Rf^(rho_j, theta_i) exp(2pi * 1i * rho_j \ 
                                                                                       * (x[k],theta_i))
        endfor
        
        return f
        
    # END

**Задание 3.1.** Реализуйте формулу обратной проекции, используя формулу выше и соответствующий псевдокод алгоритма. *Совет: для ускорения вычислений старайтесь использовать матричные операции в NumPy и 
старайтесь избегать циклов.* *(возможно вы в курсе того, что большинство матричных функции в NumPy являются "оберткой" функций имплементированных в C, FORTRAN и существенно оптимизированных (зачастую это функции из известных  библиотек LAPACK, BLAS))*

In [2]:
# YOUR CODE HERE 

def radon2d_fbp(npixels, projections, padding):
    ...
    return None

**Задание 3.2.** Протестируйте ваш алгоритм восстановления на двух фантомах из прошлого задания. То есть вычислите  преобразования Радона используя функцию `radon2d(image, ntheta, nshift, radius=1.0)` из прошлого задания и примените формулу обратной проекции. В качестве параметров, возьмите разрешение изображений $64\times 64$, $N_s=256$, $N_\theta=512$. (Если восстановление выглядит неудовлетворительно, попробуйте увеличить количество лучевых данных) Что происходит, когда лучевых данных мало (относительно разрешения изображения), какие артефакты вы видите? 
Как это связано с теоремой Найквиста-Шеннона-Котельникова?

Какая часть вашего алгоритма работает дольше всего? Для вычисления времени выполнения части кода используйте функцию `perf_counter()` из модуля `time`:

        tic = time.perf_counter()
        ....
        toc = time.perf_counter()
        elapsed_time = toc - tic # in seconds


In [ ]:
def circ_phantom(N):
    lin = np.linspace(-1,1, N)
    [XX, YY] = np.meshgrid(lin, lin)
    RR = np.sqrt(XX**2 + YY**2)
    circ_image = np.zeros((N,N))
    circ_image[RR < 0.5] = 1.
    circ_image[RR < 0.25] = 0.
    
    return circ_image

# 1. phantoms 

N = 32
phantom1 = circ_phantom(N) # circular layer Phantom
phantom2 = np.reshape(tomopy.misc.phantom.shepp2d(N), (N, N)) # Shepp-Logan Phantom

fig, axs = plt.subplots(1, 2, figsize=(12, 8), sharey=True)
axs[0].imshow(phantom1)
axs[0].set_title('phantom 1')
axs[1].imshow(phantom2)
axs[1].set_title('phantom 2')
plt.show()

# 2. computation of projection data

projection1 = radon2d(phantom1, 512, 256, 1.0)
projection2 = radon2d(phantom2, 512, 256, 1.0)


In [ ]:
# YOUR CODE HERE

reconstruction1 = None
reconstruction2 = None

fig, axs = plt.subplots(1, 2, figsize=(12, 8), sharey=True)
axs[0].imshow(reconstruction1_32)
axs[0].set_title('FBP-reconstruction phantom 1')
axs[1].imshow(reconstruction2_32)
axs[1].set_title('FBP-reconstruction phantom 2')
plt.show()


**Ответы на вопросы из Задания 3.2.:** ВАШ ОТВЕТ

**Задание 3.3.** Восстановите изображение из лучевых данных из binary-файла, который подготовлен заранее. Выполните строку ниже, которая загрузит лучевые данные в вашу текущую папку и примените к данным алгоритм обратной проекции тем самым восстановите изображение. 

Формат файла: 

    binary file N_theta = 512, N_s=256, floats of 4 bytes each, binary format - little endian 
    
Бинарные файлы в NumPy можно загружать при помощи команды:

    np.fromfile(file, dtype=float, count=-1, sep=", offset=0)
    

In [ ]:
# library for file download
import wget

filename = 'imag' + str(my_id) + '.bin'
address = 'https://raw.github.com/fedor-goncharov/pdo-tomography-course/master/seminar-materials/seminar-2/'
url = address + filename
output_path = './'
wget.download(url, output_path)

In [ ]:
# read projections from the downloaded binary file

ntheta = 512
nshift = 256
projections_exterior_data = np.reshape(np.fromfile(filename, dtype=np.float), (ntheta, nshift))

# YOUR CODE HERE


## 4. Спектральные методы. Сглаживающие фильтры

### 4.1 Спектральная реализация формулы обратной проекции  

Формулу FBP можно реализовать множеством различных способов - все они в итоге должны давать одинаковые восстановления, но могут существенно различатся по скорости вычислений и по требованиям к памяти. 

В параграфе 3 при реализации FBP мы использовали следующий порядок вычислений: 

 1. Вычисление $\widehat{Rf}$ (очень быстро)
 2. Применение фильтра $\sim |\rho|$ (очень быстро)
 3. Обратное проецирование данных - вычисление суммы интеграла Фурье (медленно)

Но можно провести восстановления и в обратном порядке (обратная проекция, потом фильтрация):

 1. Вычисление обратной проекции $R^*Rf$ (возможно не быстро)
 2. Применение спектрального фильтра $\sim |\rho|$ (быстро, так как можно реализовать через DFT)
 3. Обратное проецирование данных - вычисление суммы интеграла Фурье (быстро, так как на равномерной сетке, DFT)
 
**Замечание:** не всегда можно менять местами проецирование и фильтрацию, но в данном случае, на плоскости $R^2$ это действительно приводит к тому-же результату.
 
**Задание в этом параграфе:** В данной главе мы займемся последней реализацией FBP и сравним скорость нового алгоритма с алгоритмом  из задания 3. Главный ключ к нашей реализации - это следующая формула: 
\begin{equation}
    R^*Rf(x) = 2 \left(f \ast \dfrac{1}{|x|}\right)(x),
\end{equation}
где $R^*$ - формула обратной проекции, $R$ - преобразование Радона, $\ast$ - обозначает конволюцию функций на $R^2$.

**Упражнение (факультативно).** Докажите корректность данной формулы.

Несложно показать, что с точки зрения преобразования Фурье предыдущая формула выглядит следующим образом: 
\begin{equation}
    \mathcal{F}(R^*Rf)(\xi) = \mathcal{F}f(\xi) \cdot \dfrac{1}{|\xi|}, \, \xi \neq 0.
\end{equation}
 
Тогда мы сразу получаем следующий алгоритм для FBP:

    
         # 1. initialization
             image(x,y) = []
             filter(xi_1,xi_2) = |(xi_1, xi_2)|
             projection-data(ntheta, nshift)
         # 2. Compute adjoint from projection data
             g(x,y) = R*(projection-data)(x,y)
         # 3. Take Fourier transform of g
             Fg(xi_1, xi_2) = 
         # 4. Apply filter
             WFg(xi_1, xi_2) = Fg(xi_1, xi_2) .* filter(xi_1, xi_2) # elementwise product
         # 5. Take inverse Fourier transform
             image(x,y) = F^{-1}(WFg)(x,y)
       
             return image
       
    
**Задание 4.1** Имплементируйте псевдокод выше для восстановления сигнала по лучевым преобразованиям. В качестве оператора $R^*$ используйте функцию `adjradon2d(...)` имплементированную ниже (этот код работает намного быстрее того, что вы писали в первом задании, так как он не вычисляет матрицу для $R$). Для применения фильтра в пространстве частот можете использовать участки кода для реализации FBP из главы 3. Примените реализованный метод восстановления к лучевым данным из задания 3.2.



In [ ]:
# 1. compute adjoint
npixels = 512
dom_rad = 2.0
adjoint_exterior_data = adjradon2d(projections_exterior_data, npixels, dom_rad)

# YOUR CODE HERE

# 2. create Fourier filter

# 3. apply Fourier filter


**Задание 4.2** Сравните скорость работы нового алгоритма, с алгоритмом из задания 3.2. Для вычисления времени выполнения части кода используйте функцию `perf_counter()` из модуля `time`. Существенным ли получилось ускорение?

In [53]:
# YOUR CODE HERE 


**Замечение:** при правильном восстановлении, значения на краю изображения (например, в диапазоне [0:10, 0:10]) должны быть равны $0$ ну или близки к ним. Выполняется ли это свойство в вашем случае? Как вы можете объяснить результат в вашем случае?



In [ ]:
# YOUR CODE HERE 


**Задание 4.3.** По краям восстановленного изображения можно заметить артефакты, этому есть простое объяснение. Функция $R^*Rf$ в $R^2$ не имеет компактного носителя, так как это конволюция с функцией $1/ |x|$ с неограниченным носителем . Мы используем значения $R^*Rf$ в квадрате $N\times N$ пикселей, поэтому при использовании DFT, где неявно предполагается, что картинка - это периодический сигнал, мы получаем "разрывы" гладкости по границам изображения (в этих участках функция недифференцируема). Фильтр $\sim |\rho|$ соответствует производной (на самом деле производной от преобразования Гильберта), поэтому на границах мы получаем артефакты в виде скачков. В этом задании вам предлагается использовать "трюк", чтобы уменьшить влияние этих артефактов. Для этого, сначала восстановите $R^*Rf$ на большей картинке (например, $2N\times 2N$ пикселей), после чего домножьте (попиксельно) изображение на функцию, которая равна $1.0$ в круге вписанном в квадрат $N\times N$ пикселей и гладко убывает к нулю к краям квадрата $2N\times 2N$ (такая функция называется маской). После этого примените фильтр $\sim |\rho|$, в квадрате $N\times N$ будет находится нужное вам изображение. Идея в том, что использование маски позволяет сделать сигнал "периодичным" без разрывов, что уберёт артефакты по краям после применения фильтра. Реализуйте описанный алгоритм и проверьте его на данных из задания 3.3. (для создания маски полезной может оказаться функция [Smoothstep](https://en.wikipedia.org/wiki/Smoothstep))

In [ ]:
# YOUR CODE 

# SOLUTION


### 4.2 Спектральное сглаживание

Как мы видели на картинках из задания 3, если функции, которые мы восстанавливаем - локально-постоянные (т. е.  содержат разрывы, скачки), то даже при существенном увеличении кол-ва данных и уменьшения сеток - **на восстановлениях видны небольшие осцилляции**. Если это незаметно на двумерных изображениях, постройте графики сечений вдоль осей $Ox$, $Oy$, на них осцилляции видны лучше.

Эти осцилляции связаны с [эффектом Гиббса](https://en.wikipedia.org/wiki/Gibbs_phenomenon) - Фурье преобразование таких функций убывает медленно, поэтому наши восстановления, которые соответствуют частичному ряду Фурье (до частоты Найквиста), не включают в себя достаточно "сильно-осциллирующих" гармоник, чтобы приблизить скачок.

Хоть это и вполне "честное" восстановление (и даже оптимальное в $L^2$-норме) - такие осциляции могут быть неприятны человеческому глазу, если мы хотим видеть, например, какие-то общие контуры на изображении.

Поэтому, на практике используют процедуры, сглаживающие итоговое изображение, получая хоть и менее "оптимальное", зато более приятное для глаза. Нет общего правила, что такое оптимальное сглаживание, поэтому 
процедур сглаживания существует огромное множество - начиная от пространственно-инвариантных фильтров, и заканчивая нейросетями, сглаживающими определенные участки изображений разными фильтрами.

В данном задании мы применим самый простой вид сглаживания - **пространственно-инвариантные фильтры** с помощью преобразования Фурье.

**Задание 4.4** Добавьте в любой из ваших алгоритмов FBP $sinc$-фильтр:
\begin{equation}
    H(u, v) = \begin{cases}
        \dfrac{\sin\left(\dfrac{\pi}{\nu_{Nyq}}\sqrt{(u^2 + v^2)}\right)}
        {\dfrac{\pi}{\nu_{Nyq}}\sqrt{u^2 + v^2}}, \sqrt{u^2 + v^2} \leq \nu_{Nyq},\\
        0, \text{ иначе. }
    \end{cases}
\end{equation}

где $\nu_{Nyq}$ - частота Найквиста. 
Протестируйте ваш алгоритм на фантомах 1, 2 из задания 3.2. Видите ли вы разницу между вашим восстановленным изображением и изображениями из задания 3.2?



In [ ]:
# YOUR CODE HERE

# SOLUTION


### 5. Домашнее задание - ускорение FBP (задание факультативное но близкое к вычислениям на практике) 

**Для этого задания вам понадобится Matlab или Octave > 4.4**. 

В задании 3.2. вы могли заметить, что часть алгоритма, связанная с вычислением $\widehat{Rf}$ работает очень быстро относительно той части которая считает финальную сумму интеграла Фурье. 

Это в некотором смысле неудивительно, ведь $\widehat{Rf}$ вычисляется при помощи DFT работающего со скоростью $O(N\cdot log N)$ вместо $O(N^2)$ для массива длиной $N$. Финальная же сумма в нашей реализации считается "в лоб", и даже с ускорением путем матричных операций она вычисляется 
относительно медленно и её асимптотика равна $O(N_p \times N_s \times N_\theta$), где $N_p$ - кол-во пикселей на изображении ($N_p = N^2$ пикселей для изображения $N\times N$). Поэтому, в целом, независимо от языка программирования, последняя часть кода будет работать всегда медленно. Как мы увидим далее, в реальных медицинских приложениях важно как раз быстрое восстановление изображения, поэтому нашему алгоритму необходимо ускорение. 

Но как ускорить вычисление суммы интеграла Фурье? Чтобы ответить на этот вопрос, во-первых, надо посмотреть ещё раз на математическую задачу, которую мы решаем этим участком кода.

Участок кода вычисляет следующую сумму: 

\begin{equation}
    f(x) \approx \Delta_\varphi \Delta_\rho\sum_{i=1}^{N_\theta} \sum_{j = 1}^{N_\rho} \dfrac{|\rho_j|}{2}
    \widehat{Rf}(\rho_j, \theta_i) e^{2\pi i (x,\theta_i)\rho_j}, 
\end{equation}
где $\widehat{Rf}$ - известно.


В силу проекционной теоремы сумма выше содержит значения $\widehat{f}$ в следующих точках - рисунок слева.


<table>
<tr>
 <td> <img src="../seminar-materials/seminar-2/fourier_sampling.png" width=350 height=350> </td>
 <td> <img src="../seminar-materials/seminar-2/fourier_sampling_rectangle.png" width=430 height=450 /> </td>
</tr>
</table>

То есть функция $f$ восстанавливается из значений $\widehat{f}$ в точках лежащих на некоторой сетке. Наш алгоритм использует точки из сетки полярных координат (см. левый рисунок). Но, например, если бы нам были известны значения $\widehat{f}$ 
на равномерной сетке как на рисунке справа, то $f$ можно было также восстановить используя обратное преобразование Фурье $\mathcal{F}^{-1}$ при помощи стандартного 2D DFT. При увеличении количества точек, разницы между использованием разных сеток нет, но когда данные конечны (т. е. всегда) - **разница есть**.

  1. Слева, в сумме Римана для интеграла Фурье есть множитель $|\rho_j|$ - который является "фильтром" и усиливает      "высокие частоты". Численно это означает, что если наши данные $\widehat{Rf}(\rho, \cdot)$ содержат ошибки в области высоких частот (больших $\rho$), то применение фильтра $|\rho_j|$ усилит ошибки. Это как раз отражение нестабильности обращения преобразования Радона.
  2. Для равномерной сетки, сумма Римана бы выглядела следующим образом: 
     \begin{equation}
         f(x) \approx \Delta_x \Delta_y \sum_{m=1}^{N_x}\sum_{n=1}^{N_y} \widehat{f}(m\Delta_x, n\Delta_y)
         e^{2\pi i x_1m\Delta_x + x_2n\Delta_y}
     \end{equation}
     
     Если $x$ лежит в равномерной сетке, то есть $x=(\Delta k, \Delta l)$, то $f = \{f_{kl}\}$ и предыдущую сумму можно записать как
     \begin{equation}
         f_{kl} \approx \Delta_x \Delta_y \sum_{m=1}^{N_x}\sum_{n=1}^{N_y} \widehat{f}(m\Delta_x, n\Delta_y)
         e^{2\pi i (km\Delta\Delta_x + ln\Delta\Delta_y)} = 
         \Delta_x \Delta_y \sum_{m=1}^{N_x}\sum_{n=1}^{N_y} \hat{f}_{mn} e^{2\pi i (km\Delta\Delta_x + ln\Delta\Delta_y)}
     \end{equation}
    Можно заметить, что последняя сумма соответствует **2D DFT** от $\widehat{f}_{mn}$ - **это важно.** Зная  $\hat{f}$ на равномерной сетке можно восстановить $f$ (также на равномерной сетке) и это **работает очень быстро**.
    
**Итог**: Прямое вычисление суммы Римана в интеграле Фурье - небыстрая операция, и это следствие **неравномерности сетки в пространстве частот**. В случае равномерных сеток - вычисления могут существенно ускорены с использованием 2D DFT.

Чтобы решить эту проблему, на практике поступают следующим образом: 

 1. Интерполируют $\widehat{f}$ на равномерную сетку
 2. Восстанавливают $f$ при помощи 2D DFT
     
В данном задании мы воспользуемся библиотекой NUFFT (Non-uniform Fast Fourier Transform),  в которой уже имплементированы пункты 1, 2. Для этой библиотеки есть интерфейсы в C и Matlab/Octave.

**Задание-доклад.** Инструкция, как работает NUFFT находится здесь [NUFFT Home Page](https://www-user.tu-chemnitz.de/~potts/nfft/index.php). Сделать доклад на 20-30 минут (можно и дольше, если будет пожелание аудитории), о том как работает NUFFT с 1-2 любыми примерами.

**Задача-по-желанию.** Какая асимпототика вычисления суммы интеграла Фурье по методу NUFFT? Лучше ли она нашего метода вычисления суммы "в лоб"? Для ответа используйте документацию к библиотеке [NUFFT Documentation](https://www-user.tu-chemnitz.de/~potts/nfft/doc.php).


**Установка NUFFT для Octave**

   1. Скачайте скомпилированные mex-файлы [NUFFT](https://www-user.tu-chemnitz.de/~potts/nfft/) для Маtlab/Octave в отдельную папку; в файле .octaverc пропишите директорию к mex-файлам;
   2. Скачайте скрипты rtft2d.m, nfft_reconstruct_2d.m (rtft2d - вычисляет 1D преобразования Фурье от проекционных данных, nfft_reconstruct_2d.m - вычисляет сумму интеграла Фурье по методу NUFFT)
   

**Запуск**

   1. `[nodes, values, jacobian_weights] = rtft2d(projections, ntheta, nshift, rsupp, padd)` - вычисляет одномерные преобразования Фурье от ваших проекционных данных
   2. `reconstruction = nfft_reconstruct_2d(size, nodes, values, jacobian_weights)` - вычисляет сумму интеграла Фурье и возвращает восстановленную функцию по формуле FBP

**Задание 5.1** Установите вышеперечисленные программы и запустите их на примерах из данного задания. Реализуйте sinc-фильтр Найквиста из задания 4.4. Пришлите картинки с восстановлениями (со сглаживающим фильтром и без него), и с вашим кодом на почту tomography-course-mipt@gmail.com. 

### 6. Небольшое заключение


В данном задании, основная идея была показать, как много можно всего сделать уже с одной формулой обращения, 
а также сколько новых интересных вопросов (инженерных и теоретических) естественно возникают в процессе реализации. 

Соответственно, если вас заинтересовали какие-либо темы, либо что-то показалось недостаточно раскрытым, ваши комментарии только приветствуются (с ними можно обращаться лекторам непосредственно, либо писать на почту). 